## Aufgabe 1

In [74]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

In [75]:
# Load the data
data = pd.read_csv("data.txt", sep="\t")
genes = data.iloc[:, 0]
expression_vectors = data.iloc[:, 1:].values


In [76]:
# Farthest First Traversal for k-Center Clustering
def farthest_first_traversal(data, k):
    n = data.shape[0]
    centers = [0]  # Start with the first data point
    while len(centers) < k:
        # Calculate minimum distances from existing centers
        distances = np.array([min(euclidean(data[i], data[c]) for c in centers) for i in range(n)])
        # Select the farthest point
        farthest = np.argmax(distances)
        centers.append(farthest)
    return centers

In [77]:
# Assign points to the nearest center
def assign_to_clusters(data, centers):
    clusters = {i: [] for i in range(len(centers))}
    for i in range(len(data)):
        distances = [euclidean(data[i], data[c]) for c in centers]
        nearest = np.argmin(distances)
        clusters[nearest].append(i)
    return clusters

In [78]:
# Assign points to the nearest center
def assign_to_clusters(data, centers):
    clusters = {i: [] for i in range(len(centers))}
    for i in range(len(data)):
        distances = [euclidean(data[i], data[c]) for c in centers]
        nearest = np.argmin(distances)
        clusters[nearest].append(i)
    return clusters

In [79]:
# Perform clustering
k = 6
center_indices = farthest_first_traversal(expression_vectors, k)
clusters = assign_to_clusters(expression_vectors, center_indices)
with open("fft.txt", "w") as f:
    for cluster_id, members in sorted(clusters.items()):
        # Convert all gene names to strings and handle non-string values
        gene_names = sorted(str(genes.iloc[m]) for m in members if not pd.isna(genes.iloc[m]))
        f.write(f"Cluster {cluster_id}:\n{' '.join(gene_names)}\n")

## Aufgabe 2

Extremwerte oder Ausreißer haben einen großen Einfluss auf das kk-Center-Clustering, weil der Algorithmus versucht die maximale Distanz zwischen einem beliebigen Punkt und seinem nächsten Zentrum zu minimieren. Ein Ausreißer, der weit von allen anderen Punkten entfernt ist, erzwingt, dass ein Cluster-Zentrum speziell diesem Punkt zugeordnet wird, selbst wenn dieses Zentrum den Großteil der Daten schlecht repräsentiert. Dadurch werden die restlichen Cluster suboptimal verteilt.

## Aufgabe 3

O(n⋅k+n2)

## Aufgabe 4

K-Center fokussiert auf die Minimierung der maximalen Distanz von einem Punkt zu seinem nächsten Zentrum. Die Cluster-Zentren sind die tatsächlichen Datenpunkte. K-Center ist sehr empfindlich gegenüber Ausreißern. 

K-Means anderseits fokussiert sich auf die Minimierung der Summe der quadrierten Distanzen von jedem Punkt zu seinem Cluster-Zentrum (geometrische Mittelpunkte).  K-Means ist nicht so empfindlich wie K-Center aufgrund der Mittelung.

## Aufgabe 5

Wenn \(k=1\), das Problem reduziert sich darauf, den Mittelwert des Datensatzes zu berechnen unter verwendung der Formel:

$$
\text{Centroid} = \frac{1}{n} \sum_{i=1}^n x_i
$$

wobei \(x_i\) die Datenpunkte sind. Dies minimiert die Summe der quadrierten Abstände aller Punkte zum Zentrum, was das Ziel des \(k\)-Means-Clustering ist.

## Aufgabe 6

In [80]:
import random

# Lloyd Algorithm for k-Means Clustering
def lloyd_algorithm(data, k, max_iterations=100):
    # Initialize cluster centers randomly
    centers = data[random.sample(range(len(data)), k)]
    for _ in range(max_iterations):
        # Assign points to the nearest center
        clusters = {i: [] for i in range(k)}
        for i in range(len(data)):
            distances = [euclidean(data[i], center) for center in centers]
            nearest = np.argmin(distances)
            clusters[nearest].append(i)
        
        # Update cluster centers
        new_centers = np.array([np.mean(data[members], axis=0) if members else center for center, members in zip(centers, clusters.values())])
        if np.allclose(new_centers, centers):  # Check for convergence
            break
        centers = new_centers
    return clusters

# Perform clustering
k = 5
clusters = lloyd_algorithm(expression_vectors, k)

# Format and save the output
with open("lloyd.txt", "w") as f:
    for cluster_id, members in sorted(clusters.items()):
        gene_names = sorted(str(genes.iloc[m]) for m in members if not pd.isna(genes.iloc[m]))
        f.write(f"Cluster {cluster_id}:\n{' '.join(gene_names)}\n")


## Aufgabe 7

In [81]:
# k-Means++ Initialization
def kmeans_plus_plus_initializer(data, k):
    n = data.shape[0]
    centers = [random.randint(0, n - 1)]  # Randomly pick the first center
    while len(centers) < k:
        # Calculate the minimum distances to existing centers
        distances = np.min([np.linalg.norm(data - data[c], axis=1) for c in centers], axis=0)
        
        # Handle invalid distances by replacing zeros with a small value (epsilon)
        epsilon = 1e-10
        distances = np.where(distances == 0, epsilon, distances)
        
        # Compute probabilities for selecting the next center
        probabilities = distances**2 / np.sum(distances**2)
        
        # Ensure probabilities do not contain NaN values
        if np.any(np.isnan(probabilities)):
            raise ValueError("Probabilities contain NaN values. Check your data or initialization.")
        
        # Choose the next center
        new_center = np.random.choice(range(n), p=probabilities)
        centers.append(new_center)
    return data[centers]

# Extend Lloyd Algorithm with k-Means++ Initialization
def lloyd_plus_plus(data, k, max_iterations=100):
    centers = kmeans_plus_plus_initializer(data, k)
    return lloyd_algorithm(data, k, max_iterations)

In [82]:
# Perform clustering
clusters = lloyd_plus_plus(expression_vectors, k)

# Format and save the output
with open("lloydpp.txt", "w") as f:
    for cluster_id, members in sorted(clusters.items()):
        gene_names = sorted(str(genes.iloc[m]) for m in members if not pd.isna(genes.iloc[m]))
        f.write(f"Cluster {cluster_id}:\n{' '.join(gene_names)}\n")


## Aufgabe 8

In [86]:
import matplotlib.pyplot as plt

def plot_clusters(data, clusters, genes):
    for cluster_id, members in clusters.items():
        if len(members) == 0:
            print(f"Cluster {cluster_id} is empty. Skipping...")
            continue

        plt.figure(figsize=(10, 6))
        for member in members:
            gene_name = genes[member]  # Get the gene name
            y = data[member]  # Expression vector for the gene

            # Check for valid data
            if y.size == 0 or np.isnan(y).any():
                print(f"Skipping invalid data for gene {gene_name}")
                continue

            plt.plot(range(1, 8), np.log2(y + 1), label=gene_name)  # Plot log2 expression

        # Customize the plot
        plt.title(f"Cluster {cluster_id}")
        plt.xlabel("Time Points (R1 to R7)")
        plt.ylabel("Log2 Expression Level")
        plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1.0))  # Adjust legend position
        plt.tight_layout()

        # Save the plot to a file
        filename = f"6_{cluster_id + 1}.jpg"
        plt.savefig(filename)
        print(f"Cluster {cluster_id} plot saved as {filename}")
        plt.close()


# Generate plots
plot_clusters(expression_vectors, clusters, genes)

C:\Users\Studium\AppData\Local\Temp\ipykernel_21228\344416997.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene_name = genes[member]  # Get the gene name
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Skipping invalid data for gene 12.5
Skipping invalid data for gene 0.2
Skipping invalid data for gene 0.17
Skipping invalid data for gene 0.2
Skipping invalid data for gene 10.0
Skipping invalid data for gene 0.18
Skipping invalid data for gene 5.0
Skipping invalid data for gene 5.56
Skipping invalid data for gene 4.55
Skipping invalid data for gene 6.25
Skipping invalid data for gene 7.14
Skipping invalid data for gene 4.35
Skipping invalid data for gene 5.26
Skipping invalid data for gene 5.26
Skipping invalid data for gene 0.19
Skipping invalid data for gene 0.18
Skipping invalid data for gene 6.25
Skipping invalid data for gene 7.14
Skipping invalid data for gene 5.26
Skipping invalid data for gene 5.56
Skipping invalid data for gene 5.26
Skipping invalid data for gene 5.56
Skipping invalid data for gene 0.17
Skipping invalid data for gene 2.7
Skipping invalid data for gene 0.16
Skipping invalid data for gene 0.16
Skipping invalid data for gene 0.2
Skipping invalid data for gene 2.